In [4]:
import pygame
import random
import sys
from copy import deepcopy

pygame.init()

# Constants (unchanged)
SCREEN_WIDTH = 700
SCREEN_HEIGHT = 760
BOARD_SIZE = 600
SQUARE_SIZE = BOARD_SIZE // 15

# Colors (unchanged)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 175, 0)
BLUE = (0, 0, 255)
YELLOW = (255, 255, 0)
GRAY = (200, 200, 200)

PLAYER_COLORS = [BLUE, RED, YELLOW, GREEN]
PLAYER_NAMES = ["Blue", "Red", "Yellow", "Green"]

screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Ludo Game")

# Board positions (unchanged)
HOME_POSITIONS = {
    0: [(1, 1), (1, 4), (4, 1), (4, 4)],
    1: [(1, 10), (1, 13), (4, 10), (4, 13)],
    2: [(10, 1), (10, 4), (13, 1), (13, 4)],
    3: [(10, 10), (10, 13), (13, 10), (13, 13)]
}

START_POSITIONS = {
    0: (1, 6),
    1: (6, 13),
    2: (8, 1),
    3: (13, 8)
}

PATHS = {
    0: [(1, 6), (2, 6), (3, 6), (4, 6), (5, 6), (6, 5), (6, 4), (6, 3), (6, 2), (6, 1),
        (6, 0), (7, 0), (8, 0), (8, 1), (8, 2), (8, 3), (8, 4), (8, 5), (9, 6), (10, 6),
        (11, 6), (12, 6), (13, 6), (14, 6), (14, 7), (14, 8), (13, 8), (12, 8), (11, 8),
        (10, 8), (9, 8), (8, 9), (8, 10), (8, 11), (8, 12), (8, 13), (8, 14), (7, 14),
        (6, 14), (6, 13), (6, 12), (6, 11), (6, 10), (6, 9), (5, 8), (4, 8), (3, 8),
        (2, 8), (1, 8), (0, 8), (0, 7), (1, 7), (2, 7), (3, 7), (4, 7), (5, 7), (6, 7)],
    1: [(6, 13), (6, 12), (6, 11), (6, 10), (6, 9), (5, 8), (4, 8), (3, 8), (2, 8), (1, 8),
        (0, 8), (0, 7), (0, 6), (1, 6), (2, 6), (3, 6), (4, 6), (5, 6), (6, 5), (6, 4),
        (6, 3), (6, 2), (6, 1), (6, 0), (7, 0), (8, 0), (8, 1), (8, 2), (8, 3), (8, 4),
        (8, 5), (9, 6), (10, 6), (11, 6), (12, 6), (13, 6), (14, 6), (14, 7), (14, 8),
        (13, 8), (12, 8), (11, 8), (10, 8), (9, 8), (8, 9), (8, 10), (8, 11), (8, 12),
        (8, 13), (8, 14), (7, 14), (7, 13), (7, 12), (7, 11), (7, 10), (7, 9), (7, 8)],
    2: [(8, 1), (8, 2), (8, 3), (8, 4), (8, 5), (9, 6), (10, 6), (11, 6), (12, 6), (13, 6),
        (14, 6), (14, 7), (14, 8), (13, 8), (12, 8), (11, 8), (10, 8), (9, 8), (8, 9),
        (8, 10), (8, 11), (8, 12), (8, 13), (8, 14), (7, 14), (6, 14), (6, 13), (6, 12),
        (6, 11), (6, 10), (6, 9), (5, 8), (4, 8), (3, 8), (2, 8), (1, 8), (0, 8), (0, 7),
        (0, 6), (1, 6), (2, 6), (3, 6), (4, 6), (5, 6), (6, 5), (6, 4), (6, 3), (6, 2),
        (6, 1), (6, 0), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6)],
    3: [(13, 8), (12, 8), (11, 8), (10, 8), (9, 8), (8, 9), (8, 10), (8, 11), (8, 12), (8, 13),
        (8, 14), (7, 14), (6, 14), (6, 13), (6, 12), (6, 11), (6, 10), (6, 9), (5, 8),
        (4, 8), (3, 8), (2, 8), (1, 8), (0, 8), (0, 7), (0, 6), (1, 6), (2, 6), (3, 6),
        (4, 6), (5, 6), (6, 5), (6, 4), (6, 3), (6, 2), (6, 1), (6, 0), (7, 0), (8, 0),
        (8, 1), (8, 2), (8, 3), (8, 4), (8, 5), (9, 6), (10, 6), (11, 6), (12, 6), (13, 6),
        (14, 6), (14, 7), (13, 7), (12, 7), (11, 7), (10, 7), (9, 7), (8, 7)]
}

class Token:
    def __init__(self, player_id, token_id):
        self.player_id = player_id
        self.token_id = token_id
        self.position = HOME_POSITIONS[player_id][token_id]
        self.is_at_home = True
        self.is_completed = False
        self.path_position = -1

    def draw(self):
        x, y = self.position
        screen_x = (x * SQUARE_SIZE) + (SQUARE_SIZE // 2) + 50
        screen_y = (y * SQUARE_SIZE) + (SQUARE_SIZE // 2) + 50
        pygame.draw.circle(screen, PLAYER_COLORS[self.player_id], (screen_x, screen_y), SQUARE_SIZE // 3)
        pygame.draw.circle(screen, WHITE, (screen_x, screen_y), SQUARE_SIZE // 4)
        font = pygame.font.SysFont(None, 24)
        number = font.render(str(self.token_id + 1), True, BLACK)
        screen.blit(number, (screen_x - 5, screen_y - 8))

    def move(self, steps):
        if self.is_completed:
            return False
        if self.is_at_home:
            if steps == 6:
                self.is_at_home = False
                self.path_position = 0
                self.position = PATHS[self.player_id][0]
                return True
            return False
        new_position = self.path_position + steps
        if new_position >= len(PATHS[self.player_id]):
            if new_position == len(PATHS[self.player_id]):
                self.is_completed = True
                self.position = (7, 7)
                return True
            return False
        self.path_position = new_position
        self.position = PATHS[self.player_id][new_position]
        return True

class Game:
    def __init__(self, num_players=4, num_ai_players=0):
        self.num_players = num_players
        self.num_ai_players = num_ai_players
        self.ai_players = set(range(num_players - num_ai_players, num_players))
        self.current_player = 0
        self.dice_value = 0
        self.dice_rolled = False
        self.tokens = []
        self.selected_token = None
        self.consecutive_sixes = 0
        self.ai_dice_display_time = 0
        
        for player in range(num_players):
            for token_id in range(4):
                token = Token(player, token_id)
                self.tokens.append(token)

    def roll_dice(self):
        if not self.dice_rolled:
            self.dice_value = random.randint(1, 6)
            self.dice_rolled = True
            if self.dice_value == 6:
                self.consecutive_sixes += 1
                if self.consecutive_sixes == 3:
                    self.dice_rolled = False
                    self.consecutive_sixes = 0
                    self.next_player()
                    return False
            else:
                self.consecutive_sixes = 0
            return True
        return False

    def next_player(self):
        self.current_player = (self.current_player + 1) % self.num_players
        self.dice_rolled = False
        self.selected_token = None
        self.consecutive_sixes = 0
        self.ai_dice_display_time = 0

    def select_token(self, position):
        x, y = position
        x = (x - 50) // SQUARE_SIZE
        y = (y - 50) // SQUARE_SIZE
        for token in self.tokens:
            if token.player_id == self.current_player and not token.is_completed:
                tx, ty = token.position
                if abs(x - tx) <= 0.5 and abs(y - ty) <= 0.5:
                    self.selected_token = token
                    return True
        return False

    def move_selected_token(self):
        if self.selected_token and self.dice_rolled:
            old_position = self.selected_token.position
            if self.selected_token.move(self.dice_value):
                self.check_captures(old_position)
                self.selected_token = None
                if self.dice_value != 6:
                    self.next_player()
                else:
                    self.dice_rolled = False
                return True
            return False
        return False

    def check_captures(self, old_position):
        if not self.selected_token:
            return
        safe_positions = [(6, 1), (1, 6), (6, 13), (13, 6), (13, 8), (8, 13), (8, 1), (1, 8)]
        current_pos = self.selected_token.position
        for token in self.tokens:
            if (token != self.selected_token and 
                token.player_id != self.selected_token.player_id and 
                not token.is_at_home and 
                not token.is_completed and 
                token.position == current_pos
                and current_pos not in safe_positions):
                token.is_at_home = True
                token.path_position = -1
                token.position = HOME_POSITIONS[token.player_id][token.token_id]

    def has_valid_move(self):
        if not self.dice_rolled:
            return False
        for token in self.tokens:
            if token.player_id == self.current_player and not token.is_completed:
                if token.is_at_home and self.dice_value == 6:
                    return True
                elif not token.is_at_home:
                    new_pos = token.path_position + self.dice_value
                    if new_pos <= len(PATHS[token.player_id]):
                        return True
        return False

    def get_movable_tokens(self):
        home_tokens = []
        active_tokens = []
        for token in self.tokens:
            if token.player_id == self.current_player and not token.is_completed:
                if token.is_at_home and self.dice_value == 6:
                    home_tokens.append(token)
                elif not token.is_at_home:
                    new_pos = token.path_position + self.dice_value
                    if new_pos <= len(PATHS[token.player_id]):
                        active_tokens.append(token)
        return home_tokens, active_tokens

    def check_winner(self):
        for player in range(self.num_players):
            completed = 0
            for token in self.tokens:
                if token.player_id == player and token.is_completed:
                    completed += 1
            if completed == 4:
                return player
        return None

    def evaluate_state(self, game_state, player):
        score = 0
        tokens_at_home = 0
        tokens_near_finish = 0
        for token in game_state.tokens:
            if token.player_id == player:
                if token.is_completed:
                    score += 100
                elif not token.is_at_home:
                    distance_to_finish = len(PATHS[token.player_id]) - token.path_position
                    score += token.path_position * 2
                    if distance_to_finish <= 6:
                        tokens_near_finish += 1
                        score += 20
                else:
                    tokens_at_home += 1
                    score -= 10
            else:
                if token.is_completed:
                    score -= 50
                elif not token.is_at_home:
                    score -= token.path_position
        if tokens_at_home > 2 and game_state.dice_value == 6:
            score += 30
        if tokens_near_finish > 0 and tokens_at_home > 1:
            score -= 15
        return score

    def simulate_move(self, token, steps):
        sim_token = deepcopy(token)
        if sim_token.is_at_home and steps == 6:
            sim_token.is_at_home = False
            sim_token.path_position = 0
            sim_token.position = PATHS[sim_token.player_id][0]
            return sim_token
        elif not sim_token.is_at_home:
            new_position = sim_token.path_position + steps
            if new_position < len(PATHS[sim_token.player_id]):
                sim_token.path_position = new_position
                sim_token.position = PATHS[sim_token.player_id][new_position]
                return sim_token
            elif new_position == len(PATHS[sim_token.player_id]):
                sim_token.is_completed = True
                sim_token.position = (7, 7)
                return sim_token
        return None

    def expectimax(self, depth, player, game_state, alpha, beta, is_max):
        if depth == 0 or game_state.check_winner() is not None:
            return self.evaluate_state(game_state, player)

        if is_max:
            best_value = float('-inf')
            home_tokens, active_tokens = game_state.get_movable_tokens()
            possible_moves = active_tokens + (home_tokens[:1] if home_tokens else [])
            if not possible_moves:
                return 0
            for move in possible_moves:
                new_state = deepcopy(game_state)
                for token in new_state.tokens:
                    if (token.player_id == move.player_id and 
                        token.token_id == move.token_id):
                        simulated_token = self.simulate_move(token, new_state.dice_value)
                        if simulated_token:
                            token.position = simulated_token.position
                            token.path_position = simulated_token.path_position
                            token.is_at_home = simulated_token.is_at_home
                            token.is_completed = simulated_token.is_completed
                            next_player = player if new_state.dice_value == 6 else (player + 1) % self.num_players
                            value = self.expectimax(depth - 1, next_player, new_state, alpha, beta, False)
                            best_value = max(best_value, value)
                            alpha = max(alpha, best_value)
                            if beta <= alpha:
                                break
            return best_value
        else:
            expected_value = 0
            for dice in range(1, 7):
                new_state = deepcopy(game_state)
                new_state.dice_value = dice
                new_state.dice_rolled = True
                value = self.expectimax(depth - 1, player, new_state, alpha, beta, True)
                expected_value += value / 6.0
            return expected_value

    def choose_best_move(self):
        if not self.dice_rolled or not self.has_valid_move():
            return None

        home_tokens, active_tokens = self.get_movable_tokens()
        possible_moves = active_tokens + (home_tokens[:1] if home_tokens else [])
        
        if not possible_moves:
            return None

        best_move = None
        best_value = float('-inf')
        for move in possible_moves:
            new_state = deepcopy(self)
            for token in new_state.tokens:
                if (token.player_id == move.player_id and 
                    token.token_id == move.token_id):
                    simulated_token = self.simulate_move(token, self.dice_value)
                    if simulated_token:
                        token.position = simulated_token.position
                        token.path_position = simulated_token.path_position
                        token.is_at_home = simulated_token.is_at_home
                        token.is_completed = simulated_token.is_completed
                        value = self.expectimax(2, self.current_player, new_state, float('-inf'), float('inf'), True)
                        if value > best_value:
                            best_value = value
                            best_move = move
        return best_move

    def ai_make_move(self):
        if not self.dice_rolled:
            self.roll_dice()
            self.ai_dice_display_time = pygame.time.get_ticks()
            return

        if pygame.time.get_ticks() - self.ai_dice_display_time < 1000:
            return

        best_move = self.choose_best_move()
        if best_move:
            self.selected_token = best_move
            self.move_selected_token()
            pygame.time.wait(1000)
        else:
            pygame.time.wait(1000)
            self.next_player()

    def draw(self, winner=None):
        if winner is None:
            # Draw the game board and tokens only if there’s no winner
            draw_board()
            for token in self.tokens:
                token.draw()
            if self.selected_token:
                x, y = self.selected_token.position
                screen_x = (x * SQUARE_SIZE) + (SQUARE_SIZE // 2) + 50
                screen_y = (y * SQUARE_SIZE) + (SQUARE_SIZE // 2) + 50
                pygame.draw.circle(screen, WHITE, (screen_x, screen_y), SQUARE_SIZE // 2, 2)
            
            font = pygame.font.SysFont(None, 32)
            text_y = BOARD_SIZE + 70

            player_text = f"Current Player: {PLAYER_NAMES[self.current_player]}"
            if self.current_player in self.ai_players:
                player_text += " (AI)"
            player_surface = font.render(player_text, True, PLAYER_COLORS[self.current_player])
            screen.blit(player_surface, (50, text_y))
            
            if self.dice_rolled:
                dice_text = f"Dice Outcome: {self.dice_value}"
                dice_surface = font.render(dice_text, True, BLACK)
                screen.blit(dice_surface, (50, text_y + 30))
            
            if self.current_player not in self.ai_players:
                pygame.draw.rect(screen, GRAY, (SCREEN_WIDTH - 150, SCREEN_HEIGHT - 50, 120, 40))
                button_text = font.render("Roll Dice", True, BLACK)
                screen.blit(button_text, (SCREEN_WIDTH - 140, SCREEN_HEIGHT - 45))
            
            if self.dice_rolled and not self.has_valid_move():
                info_text = "No valid moves. Turn ends."
                text_surface = font.render(info_text, True, BLACK)
                screen.blit(text_surface, (50, text_y + 60))
            elif self.dice_value == 6 and self.dice_rolled:
                info_text = "Rolled a 6! Move or roll again."
                text_surface = font.render(info_text, True, BLACK)
                screen.blit(text_surface, (50, text_y + 60))
        else:
            # Draw the winner announcement
            screen.fill(WHITE)  # Clear the screen before drawing winner
            font = pygame.font.SysFont(None, 64)
            win_text = f"{PLAYER_NAMES[winner]} wins!"
            text_surface = font.render(win_text, True, PLAYER_COLORS[winner])
            text_rect = text_surface.get_rect(center=(SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2))
            
            overlay = pygame.Surface((SCREEN_WIDTH, SCREEN_HEIGHT), pygame.SRCALPHA)
            overlay.fill((255, 255, 255, 175))
            screen.blit(overlay, (0, 0))
            screen.blit(text_surface, text_rect)
            
            font = pygame.font.SysFont(None, 32)
            restart_text = "Click anywhere to restart"
            restart_surface = font.render(restart_text, True, BLACK)
            restart_rect = restart_surface.get_rect(center=(SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2 + 50))
            screen.blit(restart_surface, restart_rect)
        
        pygame.display.flip()

def draw_board():
    screen.fill(WHITE)
    offset_x = 50
    offset_y = 50
    
    pygame.draw.rect(screen, BLUE, (offset_x, offset_y, SQUARE_SIZE * 6, SQUARE_SIZE * 6))
    pygame.draw.rect(screen, WHITE, (offset_x + SQUARE_SIZE, offset_y + SQUARE_SIZE, SQUARE_SIZE * 4, SQUARE_SIZE * 4))
    
    pygame.draw.rect(screen, YELLOW, (offset_x + SQUARE_SIZE * 9, offset_y, SQUARE_SIZE * 6, SQUARE_SIZE * 6))
    pygame.draw.rect(screen, WHITE, (offset_x + SQUARE_SIZE * 10, offset_y + SQUARE_SIZE, SQUARE_SIZE * 4, SQUARE_SIZE * 4))
    
    pygame.draw.rect(screen, RED, (offset_x, offset_y + SQUARE_SIZE * 9, SQUARE_SIZE * 6, SQUARE_SIZE * 6))
    pygame.draw.rect(screen, WHITE, (offset_x + SQUARE_SIZE, offset_y + SQUARE_SIZE * 10, SQUARE_SIZE * 4, SQUARE_SIZE * 4))
    
    pygame.draw.rect(screen, GREEN, (offset_x + SQUARE_SIZE * 9, offset_y + SQUARE_SIZE * 9, SQUARE_SIZE * 6, SQUARE_SIZE * 6))
    pygame.draw.rect(screen, WHITE, (offset_x + SQUARE_SIZE * 10, offset_y + SQUARE_SIZE * 10, SQUARE_SIZE * 4, SQUARE_SIZE * 4))
    
    # Define the four center triangles with correct colors
    pygame.draw.polygon(screen, BLUE, [(offset_x + SQUARE_SIZE * 6, offset_y + SQUARE_SIZE * 6),
                                    (offset_x + SQUARE_SIZE * 7.5, offset_y + SQUARE_SIZE * 7.5),
                                    (offset_x + SQUARE_SIZE * 6, offset_y + SQUARE_SIZE * 9)])

    pygame.draw.polygon(screen, YELLOW, [(offset_x + SQUARE_SIZE * 6, offset_y + SQUARE_SIZE * 6),
                                    (offset_x + SQUARE_SIZE * 7.5, offset_y + SQUARE_SIZE * 7.5),
                                    (offset_x + SQUARE_SIZE * 9, offset_y + SQUARE_SIZE * 6)])

    pygame.draw.polygon(screen, GREEN, [(offset_x + SQUARE_SIZE * 9, offset_y + SQUARE_SIZE * 6),
                                        (offset_x + SQUARE_SIZE * 7.5, offset_y + SQUARE_SIZE * 7.5),
                                        (offset_x + SQUARE_SIZE * 9, offset_y + SQUARE_SIZE * 9)])

    pygame.draw.polygon(screen,RED, [(offset_x + SQUARE_SIZE * 6, offset_y + SQUARE_SIZE * 9),
                                        (offset_x + SQUARE_SIZE * 7.5, offset_y + SQUARE_SIZE * 7.5),
                                        (offset_x + SQUARE_SIZE * 9, offset_y + SQUARE_SIZE * 9)])

    
    for i in range(16):
        pygame.draw.line(screen, BLACK, (offset_x, offset_y + i * SQUARE_SIZE), 
                        (offset_x + BOARD_SIZE, offset_y + i * SQUARE_SIZE), 1)
        pygame.draw.line(screen, BLACK, (offset_x + i * SQUARE_SIZE, offset_y), 
                        (offset_x + i * SQUARE_SIZE, offset_y + BOARD_SIZE), 1)
    
    for i in range(1, 6):
        pygame.draw.rect(screen, BLUE, (offset_x + SQUARE_SIZE * i, offset_y + SQUARE_SIZE * 7, SQUARE_SIZE, SQUARE_SIZE))
        pygame.draw.rect(screen, RED, (offset_x + SQUARE_SIZE * 7, offset_y + SQUARE_SIZE * (14 - i), SQUARE_SIZE, SQUARE_SIZE))
        pygame.draw.rect(screen, YELLOW, (offset_x + SQUARE_SIZE * 7, offset_y + SQUARE_SIZE * i, SQUARE_SIZE, SQUARE_SIZE))
        pygame.draw.rect(screen, GREEN, (offset_x + SQUARE_SIZE * (14 - i), offset_y + SQUARE_SIZE * 7, SQUARE_SIZE, SQUARE_SIZE))
    
    for player in range(4):
        for pos in HOME_POSITIONS[player]:
            x, y = pos
            center_x = offset_x + (x * SQUARE_SIZE) + (SQUARE_SIZE // 2)
            center_y = offset_y + (y * SQUARE_SIZE) + (SQUARE_SIZE // 2)
            pygame.draw.circle(screen, WHITE, (center_x, center_y), SQUARE_SIZE // 3)
    
    font = pygame.font.SysFont(None, 36)
    for player, pos in START_POSITIONS.items():
        x, y = pos
        screen_x = (x * SQUARE_SIZE) + (SQUARE_SIZE // 2) + offset_x
        screen_y = (y * SQUARE_SIZE) + (SQUARE_SIZE // 2) + offset_y
        star = font.render("★", True, PLAYER_COLORS[player])
        star_rect = star.get_rect(center=(screen_x, screen_y))
        screen.blit(star, star_rect)
    
    pygame.draw.rect(screen, WHITE, (offset_x + SQUARE_SIZE * 7, offset_y + SQUARE_SIZE * 7, SQUARE_SIZE, SQUARE_SIZE))
    finish_text = font.render("⚑", True, BLACK)
    finish_rect = finish_text.get_rect(center=(offset_x + SQUARE_SIZE * 7.5, offset_y + SQUARE_SIZE * 7.5))
    screen.blit(finish_text, finish_rect)
    
    safe_positions = [(6, 1), (1, 6), (6, 13), (13, 6), (13, 8), (8, 13), (8, 1), (1, 8)]
    for x, y in safe_positions:
        screen_x = offset_x + (x * SQUARE_SIZE) + (SQUARE_SIZE // 2)
        screen_y = offset_y + (y * SQUARE_SIZE) + (SQUARE_SIZE // 2)
        pygame.draw.circle(screen, BLACK, (screen_x, screen_y), 5)

def check_dice_button(pos):
    x, y = pos
    button_rect = pygame.Rect(SCREEN_WIDTH - 150, SCREEN_HEIGHT - 50, 120, 40)
    return button_rect.collidepoint(x, y)

def determine_first_player():
    rolls = []
    for i in range(4):
        rolls.append(random.randint(1, 6))
    return rolls.index(max(rolls))

def get_num_ai_players():
    font = pygame.font.SysFont(None, 48)
    input_active = True
    num_ai = ""
    
    while input_active:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_RETURN and num_ai.isdigit():
                    num = int(num_ai)
                    if 0 <= num <= 4:
                        input_active = False
                elif event.key == pygame.K_BACKSPACE:
                    num_ai = num_ai[:-1]
                elif event.unicode.isdigit():
                    num_ai += event.unicode
        
        screen.fill(WHITE)
        prompt = font.render("Enter number of AI players (0-4): " + num_ai, True, BLACK)
        screen.blit(prompt, (50, SCREEN_HEIGHT // 2))
        pygame.display.flip()
    
    return int(num_ai)

def main():
    num_ai_players = get_num_ai_players()
    first_player = determine_first_player()
    game = Game(num_players=4, num_ai_players=num_ai_players)
    game.current_player = first_player
    
    running = True
    winner = None
    
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.MOUSEBUTTONDOWN:
                if event.button == 1:
                    pos = pygame.mouse.get_pos()
                    if winner is not None:
                        num_ai_players = get_num_ai_players()
                        first_player = determine_first_player()
                        game = Game(num_players=4, num_ai_players=num_ai_players)
                        game.current_player = first_player
                        winner = None
                    elif game.current_player not in game.ai_players:
                        if check_dice_button(pos):
                            game.roll_dice()
                        elif game.dice_rolled and game.select_token(pos):
                            game.move_selected_token()

        if winner is None and game.current_player in game.ai_players:
            game.ai_make_move()

        if winner is None:
            winner = game.check_winner()

        game.draw(winner)  # Pass winner to draw method
        
        if game.dice_rolled and not game.has_valid_move() and winner is None:
            pygame.time.wait(1500)
            game.next_player()
        
        pygame.time.Clock().tick(60)
    
    pygame.quit()
    sys.exit()

if __name__ == "__main__":
    main()

SystemExit: 